In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir( os.path.join('..', 'notebook_format') )
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import numpy as np
import pandas as pd

# 1. magic to print version
# 2. magic so that the notebook will reload external python modules
%load_ext watermark
%load_ext autoreload 
%autoreload 2

from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import RMSprop
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,keras

Using TensorFlow backend.


Ethen 2017-03-24 10:45:47 

CPython 3.5.2
IPython 5.3.0

numpy 1.12.1
pandas 0.19.2
keras 1.1.1


# Keras Basics

Basic Keras API to build a simple multi-layer neural network.

In [3]:
n_classes = 10
n_features = 784 # mnist is a 28 * 28 image

# load the dataset and some preprocessing step that can be skipped
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, n_features)
X_test = X_test.reshape(10000, n_features)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# images takes values between 0 - 255, we can normalize it
# by dividing every number by 255
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices (one-hot encoding)
# note: you HAVE to to this step
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test  = np_utils.to_categorical(y_test , n_classes)

Basics of trainining a model:

The easiest way to build models in keras is to use `Sequential` model, which basically represents we're going to later use the `.add()` method to stack layers together in sequence to build up our network.

- We start with `Dense` (fully-connected layers), where we specify how many nodes you wish to have for the layer. Since the first layer that we're going to add is the input layer, we have to make sure that the `input_dim` parameter matches the number of features (columns) in the training set. Then after the first layer, we don't need to specify the size of the input anymore.
- Then we specify the `Activation` function for that layer, and add a `Dropout` layer if we wish.
- For the last `Dense` and `Activation` layer we need to specify the number of class as the output and softmax to tell it to output the predicted class's probability.

In [5]:
# train a model:
model = Sequential()
model.add(Dense(512, input_dim = n_features))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [6]:
# you can check the summary to check the number of parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
__________

Once your model looks good, configure its learning process with `.compile()`, where you need to specify which `optimizer` to use, and the `loss` function ( `categorical_crossentropy` is the typical one for multi-class classification) and the `metrics` to track. 

Finally, `.fit()` the model by passing in the training, validation set, the number of epochs and batch size. For the batch size, you typically specify this number to be power of 2 for computing efficiency (computers works faster for numbers that are powers of 2?)

In [7]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

n_epochs = 10
batch_size = 128 
history = model.fit(
    X_train, 
    Y_train,
    batch_size = batch_size, 
    epochs = n_epochs,
    verbose = 1, # set it to 0 if you do not want to have progess bars
    validation_data = (X_test, Y_test)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s - loss: 0.2440 - acc: 0.9243 - val_loss: 0.1003 - val_acc: 0.9672
Epoch 2/10
60000/60000 [==============================] - 8s - loss: 0.1022 - acc: 0.9694 - val_loss: 0.0855 - val_acc: 0.9752
Epoch 3/10
60000/60000 [==============================] - 8s - loss: 0.0743 - acc: 0.9776 - val_loss: 0.0784 - val_acc: 0.9776
Epoch 4/10
60000/60000 [==============================] - 7s - loss: 0.0597 - acc: 0.9819 - val_loss: 0.0754 - val_acc: 0.9777
Epoch 5/10
60000/60000 [==============================] - 7s - loss: 0.0510 - acc: 0.9852 - val_loss: 0.0760 - val_acc: 0.9811
Epoch 6/10
60000/60000 [==============================] - 7s - loss: 0.0417 - acc: 0.9872 - val_loss: 0.0841 - val_acc: 0.9804
Epoch 7/10
60000/60000 [==============================] - 7s - loss: 0.0385 - acc: 0.9880 - val_loss: 0.0790 - val_acc: 0.9817
Epoch 8/10
60000/60000 [==============================] - 10s

In [8]:
# history attribute stores the training and validation score and loss
history.history

{'acc': [0.92433333333333334,
  0.9694333333651225,
  0.97755000000000003,
  0.9818833333015442,
  0.98520000003178909,
  0.98716666663487751,
  0.98803333330154419,
  0.98983333333333334,
  0.99163333330154424,
  0.99208333330154419],
 'loss': [0.24404982871711253,
  0.1022316842118899,
  0.074254885268211362,
  0.059727695464591182,
  0.051025118251641591,
  0.041701977497587604,
  0.038540927192009987,
  0.033848565337558589,
  0.030609310701318705,
  0.028627033825792993],
 'val_acc': [0.96719999999999995,
  0.97519999999999996,
  0.97760000000000002,
  0.97770000000000001,
  0.98109999999999997,
  0.98040000000000005,
  0.98170000000000002,
  0.98119999999999996,
  0.9819,
  0.98119999999999996],
 'val_loss': [0.10030656621493399,
  0.085478710957244033,
  0.07835875841774978,
  0.075407689415267667,
  0.076002575470134615,
  0.084091279012709852,
  0.079036647132626967,
  0.087390851392178587,
  0.089359093472478113,
  0.092166959850700364]}

In [9]:
# .evaluate gives the loss and metric evaluation score for the dataset,
# here the result matches the validation set's history above
score = model.evaluate(X_test, Y_test, verbose = 0)
score

[0.092166966785416665, 0.98119999999999996]

In [10]:
# stores the weight of the model,
# it's a list, note that the length is 6 because we have 3 dense layer
# and each one has it's associated bias term
weights = model.get_weights()
print( len(weights) )

# W1 should have 784, 512 for the 784
# feauture column and the 512 the number 
# of dense nodes that we've specified
W1, b1, W2, b2, W3, b3 = weights
print(W1.shape)
print(b1.shape)

6
(784, 512)
(512,)


In [11]:
# predict the accuracy
y_pred = model.predict_classes(X_test, verbose = 0)
accuracy = np.sum(y_test == y_pred) / X_test.shape[0]
print( 'valid accuracy: %.2f' % (accuracy * 100) )

valid accuracy: 98.12


## Saving and loading the models

It is not recommended to use pickle or cPickle to save a Keras model. By saving it as a HDF5 file, we can preserve the configuartion and weights of the model.

In [12]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [13]:
# testing: predict the accuracy using the loaded model
y_pred = model.predict_classes(X_test, verbose = 0)
accuracy = np.sum(y_test == y_pred) / X_test.shape[0]
print( 'valid accuracy: %.2f' % (accuracy * 100) )

valid accuracy: 98.12


# Reference

- [Keras Documentation](http://keras.io/) 
- [Keras Documentation: mnist_mlp example](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py)
- [Keras Documentation: Saving Keras Model](http://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)